In [1]:
"""
Feature: catheter_indication
Extract from discharge notes using regex patterns
"""
import pandas as pd
import os
import re
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load discharge notes
discharge = pd.read_csv(
    os.path.join(note_path, "discharge.csv"),
    usecols=["subject_id", "hadm_id", "text"]
)

# Catheter indication pattern
discharge["catheter_indication"] = discharge["text"].str.contains(
    r"(?:urinary retention|monitor(?:ing)? urine|strict i\/o|accurate urine output|"
    r"oliguria|low urine output|aki monitoring|post[- ]op retention|diuresis)",
    flags=re.IGNORECASE,
    regex=True,
    na=False
)

# Aggregate per admission
catheter_indication_adm = (
    discharge.groupby(["subject_id", "hadm_id"])["catheter_indication"]
    .any()
    .reset_index()
)

# Merge
df = df.merge(catheter_indication_adm, on=["subject_id", "hadm_id"], how="left")

# Fill missing with False
df["catheter_indication"] = df["catheter_indication"].fillna(False).astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'catheter_indication' added. True count: {df['catheter_indication'].sum()}")
print(f"Dataset shape: {df.shape}")



C:\Users\Coditas\AppData\Local\Temp\ipykernel_19092\4134163737.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["catheter_indication"] = df["catheter_indication"].fillna(False).astype(bool)



Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 68)
Feature 'catheter_indication' added. True count: 26993
Dataset shape: (158020, 68)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "catheter_indication"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)